In [3]:
import pandas as pd

df1 = pd.read_csv("Emotion_final.csv")
df = pd.read_csv("text_emotion.csv")
df.head(5)

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [4]:
df = df[["content","sentiment"]] 
df

,content,sentiment
0,@tiffanylue i know i was listenin to bad habi...,empty
1,Layin n bed with a headache ughhhh...waitin o...,sadness
2,Funeral ceremony...gloomy friday...,sadness
3,wants to hang out with friends SOON!,enthusiasm
4,@dannycastillo We want to trade with someone w...,neutral
...,...,...
39995,@JohnLloydTaylor,neutral
39996,Happy Mothers Day All my love,love
39997,Happy Mother's Day to all the mommies out ther...,love
39998,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...,happiness


In [5]:
df_concat = df.rename(columns = {'content': 'Text', 'sentiment': 'Emotion'})
df_concat

,Text,Emotion
0,@tiffanylue i know i was listenin to bad habi...,empty
1,Layin n bed with a headache ughhhh...waitin o...,sadness
2,Funeral ceremony...gloomy friday...,sadness
3,wants to hang out with friends SOON!,enthusiasm
4,@dannycastillo We want to trade with someone w...,neutral
...,...,...
39995,@JohnLloydTaylor,neutral
39996,Happy Mothers Day All my love,love
39997,Happy Mother's Day to all the mommies out ther...,love
39998,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...,happiness


In [6]:
data_concat = pd.concat([df1,df_concat],ignore_index=True)
data_concat

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
61454,@JohnLloydTaylor,neutral
61455,Happy Mothers Day All my love,love
61456,Happy Mother's Day to all the mommies out ther...,love
61457,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...,happiness


In [7]:
data_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61459 entries, 0 to 61458
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Text     61459 non-null  object
 1   Emotion  61459 non-null  object
dtypes: object(2)
memory usage: 960.4+ KB


In [8]:
data_concat.loc[1]

Text       i can go from feeling so hopeless to so damned...
Emotion                                              sadness
Name: 1, dtype: object

In [9]:
data_concat.Emotion = data_concat.Emotion.replace(['happiness','happy','sadness','anger','worry','fear','love','surprise','fun','relief'
                                     ,'empty','enthusiasm','boredom','hate','neutral']
                                    ,[1,1,2,3,4,4,5,6,7,8,9,10,11,12,13])
data_concat.head(5)

,Text,Emotion
0,i didnt feel humiliated,2
1,i can go from feeling so hopeless to so damned...,2
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,5
4,i am feeling grouchy,3


# Pipelines (ngrame_range=(1,2), stopwords)

In [13]:
from time import time
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import SGDClassifier, LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn import svm
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB, CategoricalNB, ComplementNB, BernoulliNB
from sklearn.decomposition import FastICA, KernelPCA, TruncatedSVD, SparsePCA, NMF, FactorAnalysis, LatentDirichletAllocation
from sklearn.metrics import precision_score, recall_score, accuracy_score,f1_score
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


corpus = data_concat.Text
targets = data_concat.Emotion

stopword = stopwords.words('english')

pipe0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('logit', LogisticRegression()),
])
pipe1 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', svm.SVC(kernel='poly')),
])
pipe2 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('KNN', KNeighborsClassifier()),
])
pipe3 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svml', LinearSVC()),
])
pipe4 = Pipeline([
    ('vect', CountVectorizer(stop_words=stopword)),
    ('decision_tree', tree.DecisionTreeClassifier()),
])
parameters = {
    'stop_words': stopword,
    'ngram_range': (1,2)
    }

def run_pipes(pipes, splits=5, test_size=0.2):  
    res = defaultdict(list)
    spliter = ShuffleSplit(n_splits=splits, test_size=test_size, random_state=0)
    for idx_train, idx_test in spliter.split(corpus):
        for pipe in pipes:
            # name of the model
            name = "-".join([x[0] for x in pipe.steps])
            
            # extract datasets
            X_train = corpus[idx_train]
            X_test = corpus[idx_test]
            y_train = targets[idx_train]
            y_test = targets[idx_test]
            
            # Learn
            start = time()
            pipe.fit(X_train, y_train)
            fit_time = time() - start
            
            # predict and save results
            y = pipe.predict(X_test)
            res[name].append([
                fit_time,
                precision_score(y_test, y, average='macro'),
                pipe.score(X_test, y_test),
                recall_score(y_test, y, average='macro'),
                f1_score(y_test, y, average='macro')
            ])
    return res

def print_table(res):
    # Compute mean and std
    final = {}
    for model in res:
        arr = np.array(res[model])
        final[model] = {
            "time" : arr[:, 0].mean(),
            "precision": arr[:, 1].round(3),
            "score": arr[:, 2].round(3),
            "recall": arr[:, 3].round(3),
            "f1_score": arr[:,4].round(3)
          
        }

    D_concat = pd.DataFrame.from_dict(final, orient="index").round(3)
    #print(D)
    return D_concat



res_concat = run_pipes([pipe0, pipe1, pipe2, pipe3, pipe4], splits=1)
print_table(res_concat)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predict

,time,precision,score,recall,f1_score
tfidf-logit,20.423,[0.372],[0.516],[0.295],[0.301]
tfidf-svm,2561.826,[0.323],[0.425],[0.192],[0.182]
tfidf-KNN,0.776,[0.337],[0.352],[0.205],[0.212]
tfidf-svml,3.203,[0.336],[0.507],[0.312],[0.315]
vect-decision_tree,40.439,[0.315],[0.471],[0.302],[0.305]


# Pipelines (stopwords)

In [14]:
from time import time
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import SGDClassifier, LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn import svm
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB, CategoricalNB, ComplementNB, BernoulliNB
from sklearn.decomposition import FastICA, KernelPCA, TruncatedSVD, SparsePCA, NMF, FactorAnalysis, LatentDirichletAllocation
from sklearn.metrics import precision_score, recall_score, accuracy_score,f1_score
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


corpus = data_concat.Text
targets = data_concat.Emotion

stopword = stopwords.words('english')

pipe0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('logit', LogisticRegression()),
])
pipe1 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', svm.SVC(kernel='poly')),
])
pipe2 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('KNN', KNeighborsClassifier()),
])
pipe3 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svml', LinearSVC()),
])
pipe4 = Pipeline([
    ('vect', CountVectorizer(stop_words=stopword)),
    ('decision_tree', tree.DecisionTreeClassifier()),
])
parameters = {
    'stop_words': stopword
    }

def run_pipes(pipes, splits=5, test_size=0.2):  
    res = defaultdict(list)
    spliter = ShuffleSplit(n_splits=splits, test_size=test_size, random_state=0)
    for idx_train, idx_test in spliter.split(corpus):
        for pipe in pipes:
            # name of the model
            name = "-".join([x[0] for x in pipe.steps])
            
            # extract datasets
            X_train = corpus[idx_train]
            X_test = corpus[idx_test]
            y_train = targets[idx_train]
            y_test = targets[idx_test]
            
            # Learn
            start = time()
            pipe.fit(X_train, y_train)
            fit_time = time() - start
            
            # predict and save results
            y = pipe.predict(X_test)
            res[name].append([
                fit_time,
                precision_score(y_test, y, average='macro'),
                pipe.score(X_test, y_test),
                recall_score(y_test, y, average='macro'),
                f1_score(y_test, y, average='macro')
            ])
    return res

def print_table(res):
    # Compute mean and std
    final = {}
    for model in res:
        arr = np.array(res[model])
        final[model] = {
            "time" : arr[:, 0].mean(),
            "precision": arr[:, 1].round(3),
            "score": arr[:, 2].round(3),
            "recall": arr[:, 3].round(3),
            "f1_score": arr[:,4].round(3)
          
        }

    D_concat1 = pd.DataFrame.from_dict(final, orient="index").round(3)
    #print(D)
    return D_concat1



res_concat1 = run_pipes([pipe0, pipe1, pipe2, pipe3, pipe4], splits=1)
print_table(res_concat1)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predict

,time,precision,score,recall,f1_score
tfidf-logit,24.954,[0.372],[0.516],[0.295],[0.301]
tfidf-svm,2689.113,[0.323],[0.425],[0.192],[0.182]
tfidf-KNN,0.838,[0.337],[0.352],[0.205],[0.212]
tfidf-svml,4.300,[0.336],[0.507],[0.312],[0.315]
vect-decision_tree,41.762,[0.317],[0.472],[0.303],[0.306]


# Pipelines(function,stopwords)

In [16]:
# function lemmatization + stemming
import nltk
nltk.download('wordnet')
from nltk import WordNetLemmatizer
from nltk.stem import PorterStemmer,LancasterStemmer

z = "The girls wanted to played"


def lem_stem(z):
    lem =[]
    stemming = []
    for each in z.split() :
        lemma = WordNetLemmatizer()
        lem=[lemma.lemmatize(word)for word in z.split()]
    # Stemming
    for item in lem:
        b = PorterStemmer().stem(item)
        # = LancasterStemmer().stem(b)
        stemming.append(b)
    return stemming
print(lem_stem(z))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
['the', 'girl', 'want', 'to', 'play']


In [17]:
from time import time
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import SGDClassifier, LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn import svm
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB, CategoricalNB, ComplementNB, BernoulliNB
from sklearn.decomposition import FastICA, KernelPCA, TruncatedSVD, SparsePCA, NMF, FactorAnalysis, LatentDirichletAllocation
from sklearn.metrics import precision_score, recall_score, accuracy_score,f1_score
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


corpus = data_concat.Text
targets = data_concat.Emotion

stopword = stopwords.words('english')

pipe0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('logit', LogisticRegression()),
])
pipe1 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', svm.SVC(kernel='poly')),
])
pipe2 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('KNN', KNeighborsClassifier()),
])
pipe3 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svml', LinearSVC()),
])
pipe4 = Pipeline([
    ('vect', CountVectorizer(stop_words=stopword)),
    ('decision_tree', tree.DecisionTreeClassifier()),
])
parameters = {
    'stop_words': stopword,
    'analyzer': lem_stem
    }

def run_pipes(pipes, splits=5, test_size=0.2):  
    res = defaultdict(list)
    spliter = ShuffleSplit(n_splits=splits, test_size=test_size, random_state=0)
    for idx_train, idx_test in spliter.split(corpus):
        for pipe in pipes:
            # name of the model
            name = "-".join([x[0] for x in pipe.steps])
            
            # extract datasets
            X_train = corpus[idx_train]
            X_test = corpus[idx_test]
            y_train = targets[idx_train]
            y_test = targets[idx_test]
            
            # Learn
            start = time()
            pipe.fit(X_train, y_train)
            fit_time = time() - start
            
            # predict and save results
            y = pipe.predict(X_test)
            res[name].append([
                fit_time,
                precision_score(y_test, y, average='macro'),
                pipe.score(X_test, y_test),
                recall_score(y_test, y, average='macro'),
                f1_score(y_test, y, average='macro')
            ])
    return res

def print_table(res):
    # Compute mean and std
    final = {}
    for model in res:
        arr = np.array(res[model])
        final[model] = {
            "time" : arr[:, 0].mean(),
            "precision": arr[:, 1].round(3),
            "score": arr[:, 2].round(3),
            "recall": arr[:, 3].round(3),
            "f1_score": arr[:,4].round(3)
          
        }

    D_concat2 = pd.DataFrame.from_dict(final, orient="index").round(3)
    #print(D)
    return D_concat2



res_concat2 = run_pipes([pipe0, pipe1, pipe2, pipe3, pipe4], splits=1)
print_table(res_concat2)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predict

,time,precision,score,recall,f1_score
tfidf-logit,25.655,[0.372],[0.516],[0.295],[0.301]
tfidf-svm,2723.285,[0.323],[0.425],[0.192],[0.182]
tfidf-KNN,0.798,[0.337],[0.352],[0.205],[0.212]
tfidf-svml,3.629,[0.336],[0.507],[0.312],[0.315]
vect-decision_tree,40.556,[0.313],[0.468],[0.3],[0.304]


# Mettre les resultats des pipes dans un seule dataframe

In [19]:
print_table(res_concat2)

,time,precision,score,recall,f1_score
tfidf-logit,25.655,[0.372],[0.516],[0.295],[0.301]
tfidf-svm,2723.285,[0.323],[0.425],[0.192],[0.182]
tfidf-KNN,0.798,[0.337],[0.352],[0.205],[0.212]
tfidf-svml,3.629,[0.336],[0.507],[0.312],[0.315]
vect-decision_tree,40.556,[0.313],[0.468],[0.3],[0.304]


# TfidfVectorizer(stopwords,ngram=(1,2)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import numpy as np

x_concat = data_concat.Text
print(x_concat.shape)

y_concat = data_concat.Emotion
print(y_concat.shape)

stopword = stopwords.words('english')

vc_concat = TfidfVectorizer(stop_words=stopword, ngram_range=(1,2))
X_concat = vc_concat.fit_transform(x_concat)
print(X_concat.shape)

(61459,)
(61459,)
(61459, 376459)


In [12]:
from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import train_test_split

x_concat_train, x_concat_test, y_concat_train, y_concat_test = train_test_split(X_concat, y_concat, test_size = 0.2, random_state=0)

model_SVM_poly_concat = svm.SVC(kernel='poly')
model_SVM_poly_concat.fit(X_concat, y_concat)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [13]:
y_concat_pred=model_SVM_poly_concat.predict(x_concat_test)
print(model_SVM_poly_concat.predict(X_concat[0]))
print(model_SVM_poly_concat.predict(X_concat[3]))
print(model_SVM_poly_concat.predict(X_concat[4]))
print(model_SVM_poly_concat.predict(X_concat[39995]))
print(model_SVM_poly_concat.predict(X_concat[39999]))
print(model_SVM_poly_concat.predict(X_concat[39980]))
print(model_SVM_poly_concat.predict(X_concat[39982]))

[2]
[5]
[3]
[4]
[2]
[13]
[4]


In [14]:
print(model_SVM_poly_concat.predict(X_concat[39996]))
print(model_SVM_poly_concat.predict(X_concat[39997]))
print(model_SVM_poly_concat.predict(X_concat[39998]))
print(model_SVM_poly_concat.predict(X_concat[39986]))

[4]
[2]
[4]
[2]


In [33]:
print(X_concat[39995])
print(type(X_concat[39995]))

  (0, 41028)	0.573855757943944
  (0, 213322)	0.4366900859826896
  (0, 213337)	0.692814071651993
<class 'scipy.sparse.csr.csr_matrix'>


In [15]:
from sklearn.metrics import classification_report

print(classification_report(y_concat_test,y_concat_pred))

              precision    recall  f1-score   support

           1       0.99      1.00      0.99      2459
           2       0.99      0.99      0.99      2272
           3       1.00      1.00      1.00       616
           4       0.99      0.99      0.99      2235
           5       0.99      0.99      0.99      1109
           6       1.00      0.98      0.99       624
           7       1.00      0.98      0.99       352
           8       1.00      0.98      0.99       317
           9       1.00      0.94      0.97       158
          10       1.00      0.98      0.99       166
          11       1.00      0.97      0.99        37
          12       1.00      0.98      0.99       258
          13       0.99      0.99      0.99      1689

    accuracy                           0.99     12292
   macro avg       1.00      0.98      0.99     12292
weighted avg       0.99      0.99      0.99     12292



# Pipeline pour améliorer le model

In [10]:
from time import time
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import SGDClassifier, LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn import svm
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB, CategoricalNB, ComplementNB, BernoulliNB
from sklearn.decomposition import FastICA, KernelPCA, TruncatedSVD, SparsePCA, NMF, FactorAnalysis, LatentDirichletAllocation
from sklearn.metrics import precision_score, recall_score, accuracy_score,f1_score
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


corpus = data_concat.Text
targets = data_concat.Emotion

stopword = stopwords.words('english')

pipe0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('logit', LogisticRegression()),
])
pipe1 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svm', svm.SVC(kernel='poly')),
])
pipe2 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('KNN', KNeighborsClassifier()),
])
pipe3 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svml', LinearSVC()),
])
pipe4 = Pipeline([
    ('vect', CountVectorizer(stop_words=stopword)),
    ('decision_tree', tree.DecisionTreeClassifier()),
])
parameters = {
    'stop_words': stopword,
    'ngram_range': (1,2),
    'sublinear_tf':True, 
    'min_df':5,
    'norm':'l2'
    #'encoding':'latin-1'
    }

def run_pipes(pipes, splits=5, test_size=0.2):  
    res = defaultdict(list)
    spliter = ShuffleSplit(n_splits=splits, test_size=test_size, random_state=0)
    for idx_train, idx_test in spliter.split(corpus):
        for pipe in pipes:
            # name of the model
            name = "-".join([x[0] for x in pipe.steps])
            
            # extract datasets
            X_train = corpus[idx_train]
            X_test = corpus[idx_test]
            y_train = targets[idx_train]
            y_test = targets[idx_test]
            
            # Learn
            start = time()
            pipe.fit(X_train, y_train)
            fit_time = time() - start
            
            # predict and save results
            y = pipe.predict(X_test)
            res[name].append([
                fit_time,
                precision_score(y_test, y, average='macro'),
                pipe.score(X_test, y_test),
                recall_score(y_test, y, average='macro'),
                f1_score(y_test, y, average='macro')
            ])
    return res

def print_table(res):
    # Compute mean and std
    final = {}
    for model in res:
        arr = np.array(res[model])
        final[model] = {
            "time" : arr[:, 0].mean(),
            "precision": arr[:, 1].round(3),
            "score": arr[:, 2].round(3),
            "recall": arr[:, 3].round(3),
            "f1_score": arr[:,4].round(3)
          
        }

    D_ameliore = pd.DataFrame.from_dict(final, orient="index").round(3)
    #print(D)
    return D_ameliore



res_ameliore = run_pipes([pipe0, pipe1, pipe2, pipe3, pipe4], splits=1)
print_table(res_ameliore)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predict

,time,precision,score,recall,f1_score
tfidf-logit,22.314,[0.372],[0.516],[0.295],[0.301]
tfidf-svm,2664.365,[0.323],[0.425],[0.192],[0.182]
tfidf-KNN,0.841,[0.337],[0.352],[0.205],[0.212]
tfidf-svml,4.204,[0.336],[0.507],[0.312],[0.315]
vect-decision_tree,41.091,[0.316],[0.472],[0.303],[0.306]


On remarque que même en ajoutant des arguments comme :
- sublinear_tf = True, 
- min_df = 5,
- norm = l2  

en plus du stopwords et le ngram_range = (1,2) 
 
on obtient à peu près les mêmes scores, les plus performants sont :
- Logistic Regression
- SVM linear